# 10. 반자동 문서 스캐너 만들기

In [2]:
import cv2
import numpy as np

In [9]:
# 스캔에 사용할 이미지를 입력합니다
# img = cv2.imread("../images/sign.jpg")



## 1. 이미지를 시계 방향으로 클릭하면

In [4]:
# 포인트를 저장할 리스트를 생성합니다
# point_list = list()

## 2. 이미지에 점이 찍히고

## 3. 점과 점 사이에 라인이 연결됨
- 가능하면 라인이 마우스를 따라가도록 구현

## 4. 4개의 점을 찍으면 해당 영역이 펼쳐지도록 구현

In [56]:
import cv2
import numpy as np
img = cv2.imread("../images/sign.jpg")
temp_img = img.copy()
# 기본적인 코드 실행입니다.
name = "Scanner"
cv2.namedWindow(name)

# 포인트를 저장할 리스트를 생성합니다
point_list = []

color = (0, 0, 0)

# 최종 결과를 출력하는 함수입니다


def show_result():
    if len(point_list) == 4:
        width, height = (point_list[1][0] - point_list[0]
                         [0]), (point_list[2][1]-point_list[1][1])
        src = np.array(point_list, dtype=np.float32)
        # print(width, height)
        dst = np.array([[0, 0], [width, 0], [width, height],
                        [0, height]], dtype=np.float32)
        mat = cv2.getPerspectiveTransform(src, dst)

        result = cv2.warpPerspective(temp_img, mat, (width, height))
        cv2.imshow("result", result)

# 마우스 이벤트와 함께 실행되는 콜백함수입니다.


def onMouse(event, x, y, flags, _):
    
    # 조건을 만족하면 결과를 출력하는 함수를 실행합니다.
    if event == cv2.EVENT_LBUTTONDOWN:
        point_list.append([x, y])
        cv2.circle(img, (x, y), 3, color, cv2.FILLED, cv2.LINE_AA)
        cv2.imshow(name, img)
        if len(point_list) > 1:
            cv2.line(temp_img, point_list[-2],
                     point_list[-1], color, 2, cv2.LINE_AA)

        # 네 번째 점이 선택되면 첫 점과 마지막 점 연결
        if len(point_list) == 4:
            cv2.line(temp_img, point_list[-1],
                     point_list[0], color, 2, cv2.LINE_AA)
        show_result()


cv2.setMouseCallback(name, onMouse)
cv2.imshow(name, img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [49]:
import cv2
import numpy as np

img = cv2.imread("../images/sign.jpg")
temp_img = img.copy()  # 원본 이미지 복사본
name = "Scanner"
cv2.namedWindow(name)

point_list = []  # 선택된 점을 저장할 리스트
color = (0, 255, 0)  # 초록색
line_thickness = 2  # 선 두께

# 최종 결과를 출력하는 함수
def show_result():
    if len(point_list) == 4:  # 점이 4개일 때만 실행
        width = abs(point_list[1][0] - point_list[0][0])
        height = abs(point_list[2][1] - point_list[1][1])
        src = np.array(point_list, dtype=np.float32)
        dst = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)
        mat = cv2.getPerspectiveTransform(src, dst)

        result = cv2.warpPerspective(img, mat, (width, height))
        cv2.imshow("result", result)

# 마우스 이벤트 콜백 함수
def onMouse(event, x, y, flags, _):
    global temp_img
    temp_img = img.copy()  # 매번 원본 이미지에서 복사

    # 마우스 이동 중
    if event == cv2.EVENT_MOUSEMOVE and len(point_list) > 0:
        # 마지막 점과 마우스 위치를 임시로 연결
        cv2.line(temp_img, tuple(point_list[-1]), (x, y), color, line_thickness, cv2.LINE_AA)

    # 클릭 시
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(point_list) < 4:  # 점이 4개 이상 추가되지 않도록 제한
            point_list.append((x, y))
            cv2.circle(temp_img, (x, y), 5, color, cv2.FILLED, cv2.LINE_AA)  # 점 표시

            # 점들 간의 선 연결
            if len(point_list) > 1:
                cv2.line(temp_img, point_list[-2], point_list[-1], color, line_thickness, cv2.LINE_AA)

            # 네 번째 점이 선택되면 첫 점과 마지막 점 연결
            if len(point_list) == 4:
                cv2.line(temp_img, point_list[-1], point_list[0], color, line_thickness, cv2.LINE_AA)
                show_result()
        else:
            print("이미 4개의 점을 선택했습니다.")  # 추가적인 점 선택 시 메시지 출력

    # 점과 선 갱신
    for i, point in enumerate(point_list):
        cv2.circle(temp_img, point, 5, color, cv2.FILLED, cv2.LINE_AA)  # 선택된 점 그리기
        if i > 0:
            cv2.line(temp_img, point_list[i - 1], point, color, line_thickness, cv2.LINE_AA)

    # 화면 업데이트
    cv2.imshow(name, temp_img)

# 마우스 이벤트 설정
cv2.setMouseCallback(name, onMouse)
cv2.imshow(name, img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

img = cv2.imread("../images/sign.jpg")
temp_img = img.copy()  # 원본 이미지 복사본
overlay_img = img.copy()  # 그려진 선들을 유지하기 위한 이미지
name = "Scanner"
cv2.namedWindow(name)

point_list = []  # 선택된 점을 저장할 리스트
color = (0, 255, 0)  # 초록색
line_thickness = 2  # 선 두께

# 최종 결과를 출력하는 함수
def show_result():
    if len(point_list) == 4:  # 점이 4개일 때만 실행
        width = abs(point_list[1][0] - point_list[0][0])
        height = abs(point_list[2][1] - point_list[1][1])
        src = np.array(point_list, dtype=np.float32)
        dst = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)
        mat = cv2.getPerspectiveTransform(src, dst)

        result = cv2.warpPerspective(img, mat, (width, height))
        cv2.imshow("result", result)

# 마우스 이벤트 콜백 함수
def onMouse(event, x, y, flags, _):
    global temp_img, overlay_img
    temp_img = overlay_img.copy()  # 클릭된 선을 유지하면서 임시 선만 추가

    # 마우스 이동 중
    if event == cv2.EVENT_MOUSEMOVE and len(point_list) > 0:
        # 마지막 점과 마우스 위치를 임시로 연결
        cv2.line(temp_img, tuple(point_list[-1]), (x, y), color, line_thickness, cv2.LINE_AA)

    # 클릭 시
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(point_list) < 4:  # 점이 4개 이상 추가되지 않도록 제한
            point_list.append((x, y))
            cv2.circle(overlay_img, (x, y), 5, color, cv2.FILLED, cv2.LINE_AA)  # 점 표시

            # 점들 간의 선 연결
            if len(point_list) > 1:
                cv2.line(overlay_img, point_list[-2], point_list[-1], color, line_thickness, cv2.LINE_AA)

            # 네 번째 점이 선택되면 첫 점과 마지막 점 연결
            if len(point_list) == 4:
                cv2.line(overlay_img, point_list[-1], point_list[0], color, line_thickness, cv2.LINE_AA)
                show_result()
        else:
            print("이미 4개의 점을 선택했습니다.")  # 추가적인 점 선택 시 메시지 출력

    # 화면 업데이트
    cv2.imshow(name, temp_img)

# 마우스 이벤트 설정
cv2.setMouseCallback(name, onMouse)
cv2.imshow(name, img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [76]:
import cv2
import numpy as np

img = cv2.imread("../images/sign.jpg")
temp_img = img.copy()  # 원본 이미지 복사본
overlay_img = img.copy()  # 그려진 선들을 유지하기 위한 이미지
name = "Scanner"
cv2.namedWindow(name)

point_list = []  # 선택된 점을 저장할 리스트
color = (0, 255, 0)  # 초록색
line_thickness = 2  # 선 두께

# 최종 결과를 출력하는 함수
def show_result():
    if len(point_list) == 4:  # 점이 4개일 때만 실행
        width = abs(point_list[1][0] - point_list[0][0])
        height = abs(point_list[2][1] - point_list[1][1])
        src = np.array(point_list, dtype=np.float32)
        dst = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)
        mat = cv2.getPerspectiveTransform(src, dst)

        result = cv2.warpPerspective(img, mat, (width, height))
        cv2.imshow("result", result)

# 마우스 이벤트 콜백 함수
def onMouse(event, x, y, flags, _):
    global temp_img, overlay_img
    temp_img = overlay_img.copy()  # 클릭된 선을 유지하면서 임시 선만 추가

    # 마우스 이동 중 (4개의 점이 선택되기 전까지만 실행)
    if event == cv2.EVENT_MOUSEMOVE and len(point_list) > 0 and len(point_list) < 4:
        # 마지막 점과 마우스 위치를 임시로 연결
        cv2.line(temp_img, tuple(point_list[-1]), (x, y), color, line_thickness, cv2.LINE_AA)

    # 클릭 시
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(point_list) < 4:  # 점이 4개 이상 추가되지 않도록 제한
            point_list.append((x, y))
            cv2.circle(overlay_img, (x, y), 5, color, cv2.FILLED, cv2.LINE_AA)  # 점 표시

            # 점들 간의 선 연결
            if len(point_list) > 1:
                cv2.line(overlay_img, point_list[-2], point_list[-1], color, line_thickness, cv2.LINE_AA)

            # 네 번째 점이 선택되면 첫 점과 마지막 점 연결
            if len(point_list) == 4:
                cv2.line(overlay_img, point_list[-1], point_list[0], color, line_thickness, cv2.LINE_AA)
                show_result()

    # 화면 업데이트
    cv2.imshow(name, temp_img)

# 마우스 이벤트 설정
cv2.setMouseCallback(name, onMouse)
cv2.imshow(name, img)
cv2.waitKey(0)
cv2.destroyAllWindows()


### 점과 점 사이의 선을 그리고 싶다

In [3]:
import cv2
import numpy as np
img = cv2.imread("../images/sign.jpg")

# 기본적인 코드 실행입니다.
name = "Scanner"
cv2.namedWindow(name)

# 포인트를 저장할 리스트를 생성합니다
point_list = []

color = (0, 0, 0)

# 최종 결과를 출력하는 함수입니다


def show_result():
    if len(point_list) == 4:
        width, height = (point_list[1][0] - point_list[0]
                         [0]), (point_list[2][1]-point_list[1][1])
        src = np.array(point_list, dtype=np.float32)
        # print(width, height)
        dst = np.array([[0, 0], [width, 0], [width, height],
                        [0, height]], dtype=np.float32)
        mat = cv2.getPerspectiveTransform(src, dst)

        result = cv2.warpPerspective(img, mat, (width, height))
        cv2.imshow("result", result)

# 마우스 이벤트와 함께 실행되는 콜백함수입니다.


drawing = False


def onMouse(event, x, y, flags, _):
    global drawing
    # 조건을 만족하면 결과를 출력하는 함수를 실행합니다.
    temp_img = img.copy()
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        point_list.append([x, y])

    if drawing == True:
        prev_point = None
        for point in point_list:
            cv2.circle(temp_img, point, 3, color, cv2.FILLED, cv2.LINE_AA)

            if prev_point:
                cv2.line(temp_img, prev_point, point,
                         color, 4, cv2.LINE_AA)
            prev_point = point

        next_point = (x, y)
        if len(point_list) == 4:
            next_point = point_list[0]
            show_result()
        cv2.line(temp_img, point_list[-1],
                 next_point, color, 4, cv2.LINE_AA)

        cv2.imshow(name, temp_img)


cv2.setMouseCallback(name, onMouse)
cv2.imshow(name, img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## card

In [4]:
import cv2
import numpy as np
img = cv2.imread("../images/card.jpg")

# 기본적인 코드 실행입니다.
name = "Scanner"
cv2.namedWindow(name)

# 포인트를 저장할 리스트를 생성합니다
point_list = []

color = (0, 0, 0)

# 최종 결과를 출력하는 함수입니다


def show_result():
    if len(point_list) == 4:
        width, height = (point_list[1][0] - point_list[0]
                         [0]), (point_list[2][1]-point_list[1][1])
        src = np.array(point_list, dtype=np.float32)
        # print(width, height)
        dst = np.array([[0, 0], [width, 0], [width, height],
                        [0, height]], dtype=np.float32)
        mat = cv2.getPerspectiveTransform(src, dst)

        result = cv2.warpPerspective(img, mat, (width, height))
        cv2.imshow("result", result)

# 마우스 이벤트와 함께 실행되는 콜백함수입니다.


drawing = False


def onMouse(event, x, y, flags, _):
    global drawing
    # 조건을 만족하면 결과를 출력하는 함수를 실행합니다.
    temp_img = img.copy()
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        point_list.append([x, y])

    if drawing == True:
        prev_point = None
        for point in point_list:
            cv2.circle(temp_img, point, 3, color, cv2.FILLED, cv2.LINE_AA)

            if prev_point:
                cv2.line(temp_img, prev_point, point,
                         color, 4, cv2.LINE_AA)
            prev_point = point

        next_point = (x, y)
        if len(point_list) == 4:
            next_point = point_list[0]
            show_result()
        cv2.line(temp_img, point_list[-1],
                 next_point, color, 4, cv2.LINE_AA)

        cv2.imshow(name, temp_img)


cv2.setMouseCallback(name, onMouse)
cv2.imshow(name, img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## ground

In [5]:
import cv2
import numpy as np
img = cv2.imread("../images/ground.jpg")

# 기본적인 코드 실행입니다.
name = "Scanner"
cv2.namedWindow(name)

# 포인트를 저장할 리스트를 생성합니다
point_list = []

color = (0, 0, 0)

# 최종 결과를 출력하는 함수입니다


def show_result():
    if len(point_list) == 4:
        width, height = (point_list[1][0] - point_list[0]
                         [0]), (point_list[2][1]-point_list[1][1])
        src = np.array(point_list, dtype=np.float32)
        # print(width, height)
        dst = np.array([[0, 0], [width, 0], [width, height],
                        [0, height]], dtype=np.float32)
        mat = cv2.getPerspectiveTransform(src, dst)

        result = cv2.warpPerspective(img, mat, (width, height))
        cv2.imshow("result", result)

# 마우스 이벤트와 함께 실행되는 콜백함수입니다.


drawing = False


def onMouse(event, x, y, flags, _):
    global drawing
    # 조건을 만족하면 결과를 출력하는 함수를 실행합니다.
    temp_img = img.copy()
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        point_list.append([x, y])

    if drawing == True:
        prev_point = None
        for point in point_list:
            cv2.circle(temp_img, point, 3, color, cv2.FILLED, cv2.LINE_AA)

            if prev_point:
                cv2.line(temp_img, prev_point, point,
                         color, 4, cv2.LINE_AA)
            prev_point = point

        next_point = (x, y)
        if len(point_list) == 4:
            next_point = point_list[0]
            show_result()
        cv2.line(temp_img, point_list[-1],
                 next_point, color, 4, cv2.LINE_AA)

        cv2.imshow(name, temp_img)


cv2.setMouseCallback(name, onMouse)
cv2.imshow(name, img)
cv2.waitKey(0)
cv2.destroyAllWindows()